# Importy 

In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.feature import graycomatrix, graycoprops
import wandb
import optuna
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from joblib import Parallel, delayed
import warnings
import pickle
from IPython.display import display, HTML
from optuna.integration.wandb import WeightsAndBiasesCallback

/home/studio-lab-user/.conda/envs/plantenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Ścieżka do danych

In [2]:
TRAIN_DIR = "/home/studio-lab-user/sagemaker-studiolab-notebooks/ThesisProject/data/processed/plant_village_balanced/train"
FEATURES_CACHE = "cached_features.pkl"

# Funkcja do ekstrakcji cech z obrazu

In [4]:
def extract_features(img_path):
    features = []
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    means = cv2.mean(img)[:3]
    features.extend(means)

    glcm = graycomatrix(gray, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    for prop in ['contrast', 'correlation', 'energy', 'homogeneity']:
        features.append(graycoprops(glcm, prop)[0][0])

    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    features.append(len(contours))
    return features


# Wczytywanie danych i cache

In [5]:
def load_or_extract_features():
    if os.path.exists(FEATURES_CACHE):
        with open(FEATURES_CACHE, "rb") as f:
            return pickle.load(f)
    else:
        X, y = [], []
        class_names = sorted(os.listdir(TRAIN_DIR))
        for class_name in tqdm(class_names, desc="Loading data"):
            class_path = os.path.join(TRAIN_DIR, class_name)
            if not os.path.isdir(class_path): continue
            for fname in os.listdir(class_path):
                if fname.lower().endswith((".jpg", ".jpeg", ".png")):
                    fpath = os.path.join(class_path, fname)
                    feats = extract_features(fpath)
                    X.append(feats)
                    y.append(class_name)
        with open(FEATURES_CACHE, "wb") as f:
            pickle.dump((X, y), f)
        return X, y

X, y = load_or_extract_features()
print(f"Liczba przykładów: {len(X)}")

Loading data: 100%|██████████| 39/39 [08:03<00:00, 12.39s/it]


Liczba przykładów: 62400


# Przygotowanie danych

In [6]:
X = np.array(X)
y = LabelEncoder().fit_transform(y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Konfiguracja WandB + Optuna

In [7]:
study_name = "svm_hyperopt_study"
storage_url = f"sqlite:///{study_name}.db"
study = optuna.create_study(
    study_name=study_name,
    direction="maximize",
    storage=storage_url,
    load_if_exists=True
)

wandb_kwargs = {
    "project": "svm-feature-optuna",
    "group": "svm-optuna-research",
    "reinit": True
}
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs, metric_name="accuracy", as_multirun=True)

[I 2025-04-01 18:44:58,589] A new study created in RDB with name: svm_hyperopt_study
/tmp/ipykernel_416/1137350553.py:15: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs, metric_name="accuracy", as_multirun=True)


# Funkcja celu z @track_in_wandb


In [8]:
study_name = "svm_hyperopt_study"
storage_url = f"sqlite:///{study_name}.db"
study = optuna.create_study(
    study_name=study_name,
    direction="maximize",
    storage=storage_url,
    load_if_exists=True
)

wandb_kwargs = {
    "project": "svm-feature-optuna",
    "group": "svm-optuna-research",
    "reinit": True
}
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs, metric_name="accuracy", as_multirun=True)

[I 2025-04-01 18:44:58,655] Using an existing study with name 'svm_hyperopt_study' instead of creating a new one.
/tmp/ipykernel_416/1137350553.py:15: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs, metric_name="accuracy", as_multirun=True)


In [9]:
@wandbc.track_in_wandb()
def objective(trial):
    C = trial.suggest_float("C", 1e-3, 1e3, log=True)
    kernel = trial.suggest_categorical("kernel", ["linear", "rbf"])
    gamma = trial.suggest_float("gamma", 1e-5, 1e-1, log=True) if kernel == "rbf" else 'scale'

    model = Pipeline([
        ("scaler", StandardScaler()),
        ("svm", SVC(C=C, gamma=gamma, kernel=kernel, probability=True, max_iter=10000))
    ])

    score = cross_val_score(model, X_train, y_train, cv=3, scoring="accuracy", n_jobs=-1).mean()
    return score

/tmp/ipykernel_416/3910251723.py:1: ExperimentalWarning: track_in_wandb is experimental (supported from v3.0.0). The interface can change in the future.
  @wandbc.track_in_wandb()


# Uruchomienie optymalizacji z callbackiem (z checkpointami)

In [ ]:
study.optimize(objective, n_trials=30, callbacks=[wandbc])

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: midonik10 (midonik10-wsb-merito) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/studio-lab-user/.conda/envs/plantenv/lib/python3.11/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/studio-lab-user/.conda/envs/plantenv/lib/python3.11/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/studio-lab-user/.conda/envs/plantenv/lib/python3.11/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2025-04-01 18:49:34,790] Trial 0 finished with value: 0.4865384615384616 and parameters: {'C': 7.450164154162476, 'kernel': 'linear'}. Best is trial 0 with value: 0.4865384615384616.


C,▁
accuracy,▁
trial_number,▁
C,7.45016
accuracy,0.48654
kernel,linear
trial_number,0


[I 2025-04-01 19:01:32,279] Trial 1 finished with value: 0.026322115384615385 and parameters: {'C': 0.043896557303286586, 'kernel': 'rbf', 'gamma': 1.4266243962487302e-05}. Best is trial 0 with value: 0.4865384615384616.


C,▁
accuracy,▁
gamma,▁
trial_number,▁
C,0.0439
accuracy,0.02632
gamma,1e-05
kernel,rbf
trial_number,1


...

C,▁
accuracy,▁
trial_number,▁
C,53.01293
accuracy,0.43423
kernel,linear
trial_number,17


/home/studio-lab-user/.conda/envs/plantenv/lib/python3.11/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2025-04-01 21:07:52,494] Trial 18 finished with value: 0.46768830128205124 and parameters: {'C': 0.29720674511939105, 'kernel': 'linear'}. Best is trial 13 with value: 0.48703926282051285.


C,▁
accuracy,▁
trial_number,▁
C,0.29721
accuracy,0.46769
kernel,linear
trial_number,18


[I 2025-04-01 21:14:25,442] Trial 19 finished with value: 0.4078525641025641 and parameters: {'C': 0.009567765133614538, 'kernel': 'linear'}. Best is trial 13 with value: 0.48703926282051285.


C,▁
accuracy,▁
trial_number,▁
C,0.00957
accuracy,0.40785
kernel,linear
trial_number,19


# Podsumowanie

In [ ]:
print("# Najlepszy wynik:")
print(study.best_trial.params)
print(f"Accuracy: {study.best_trial.value:.4f}")